<img src="http://www.cidaen.es/assets/img/mCIDaeNnb.png" alt="Logo CiDAEN" align="right">




<br><br><br>
<h2><font color="#00586D" size=8>Trabajo Final de Master</font></h2>



<h1><font color="#00586D" size=5><i>Reto sobre secuenciación de célula única en linajes celulares de cáncer de mama</i></font></h1>

<br><br><br>
<div align="right">
<font color="#00586D" size=3>Alonso Felipe Ruiz</font><br>
<font color="#00586D" size=3>Máster en Ciencia de Datos e Ingeniería de Datos en la Nube</font><br>
<font color="#00586D" size=3>Universidad de Castilla-La Mancha</font>

</div>
<br>

---


<a id="Indice"></a>
<h2><font color="#00586D" size=4>Índice</font></h2>

#### <font color="#00586D"> Notebook I</font>
* [1. Introducción](#section1)
* [2. Preparación de los datos](#section2)
* [3. Exploración preliminar](#section3)

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

import pandas as pd
import numpy as np
import glob

import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
style.use('seaborn')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

---

<a id="section1"></a>
## <font color="#00586D"> 1. Contexto del proyecto</font>
<br>

A lo largo de este trabajo de final de master se utilizará una cohorte de datos obtenida a partir de la sequenciación anivel de célula única en lineas celulares de cáncer de mama bajo diferentes tratamientos. Estas lineas celulares han sido caracterizadas a nivel genómico, transcriptómico y proteómico. El juego de datos utilizado para este estudio consta de un análisis por espectometría de masas analizando 67 linajes celulares de cáncer de mama, analizando 3015 condiciones distintas, cuantificando 38 markadores para cada una de ellas.


---

<a id="section2"></a>
## <font color="#00586D"> 2. Predicción de valores perdidos</font>
<br>
El primero de los retos planteados durante el análisis de los datos consiste en la predicción de valores perdidos a partir de los reporteros de las células medidas. La imputación de valores perdidos es esencial en algunos casos, especialmente en el caso de aplicar los conocimientos básicos en clínica. Muchas veces los valores perdidos pueden ser resultado de errores experimentales, pero en otras ocasiones puede deberse a que la implementación de esa medida hace necesario aumentar el coste de la prueba o imposibilita que esta se pueda hacer con equipos convencionales. Este es por ejemplo el caso de la citometría de flujo, donde existen ciertas limitaciones en la técnica que impiden la posibilidad de medir infinitos marcadores al mismo tiempo, además de que aumentar el número de medidas que puedes tomar de manera simultanea, hace requerir de filtros de fluorescencia en el aparato que aumentan enormemente el valor de los mismo.
<br>
En el caso que nos concierne se imputarán los valores perdidos para los siguientes marcadores:
<br>
-phospho-ERK:
<br>
-phospho-Akt(Ser473):
<br>
-phospho-S6:
<br>
-phospho-HER2:
<br>
-phospho-PLCg2:
<br>

Con ese pretexto se plantea el siguiente reto, centrándose específicamente en 6 tipos celulares distintos:
<br>
-AU565:
<br>
-EFM19:
<br>
-HCC2218: 
<br>
-LY2: 
<br>
-MACLS2:
<br>
-MDAMB436:

In [ ]:
"""Lectura de los dataframes correspondientes"""
df_goldstandard = pd.read_csv("GoldStandards/sc1gold.csv") #valores reales para analizar la eficacia del modelo
df_AU565p = pd.read_csv("Single_cell_phospho/Challenge1/AU565.csv") #datasets de cada subconjunto de células para hacer la predicción
df_EFM19p = pd.read_csv("Single_cell_phospho/Challenge1/EFM19.csv")
df_HCC2218p = pd.read_csv("Single_cell_phospho/Challenge1/HCC2218.csv")
df_LY2p = pd.read_csv("Single_cell_phospho/Challenge1/LY2.csv")
df_MACLS2p = pd.read_csv("Single_cell_phospho/Challenge1/MACLS2.csv")
df_MDAMB436p = pd.read_csv("Single_cell_phospho/Challenge1/MDAMB436.csv")
df_challenge1 = (pd.concat([df_AU565p,df_EFM19p,df_HCC2218p,df_LY2p,df_MACLS2p,df_MDAMB436p]))
#selección de las columnas de interés para predecir el modelo


<a id="section11"></a>
## <font color="#00586D"> 2.2 Analisis exploratorio y preprocesamiento de los datos. </font>
<br>

En este apartado analizaremos los datos de partida, su organización y distribución, así como el procesamiento de los datos iniciales.

In [ ]:
path = str("Single_cell_phospho/complete_cell_lines/*.csv")
filenames = glob.glob(path)
dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename))
cell_phospho = pd.concat(dfs, ignore_index=True) #Leer todos los datos de fosforilación para entrenamiento
cell_phospho.head()


In [ ]:
#Eliminamos las columnas cellID y fileID que no son informativas
cell_phospho.drop(['cellID', 'fileID'], axis = 1, inplace=True)

In [ ]:
# Análisis de la distribución de las variables a predecir
fig, axes = plt.subplots(1,5, figsize=(30,3))
variables=["p.ERK", "p.Akt.Ser473.", "p.S6", "p.HER2", "p.PLCg2"]
for col, ax in enumerate(axes.flatten()):
    sns.histplot(data=cell_phospho, x=variables[col], ax=ax)
    ax.set_title(variables[col])
    ax.set_yticks([])
    ax.set_xlabel(None)
    ax.set_ylabel(None)

Vemos aquí uno de los primeros problemas que tendremos que intentar solucionar durante el desarrollo del modelo, los valores a predecir de las variables no muestran una distribución normal, además vemos que algunos valores parecen estar sobre representados, también parece haber valores extremos que pueden condicionar la capacidad de predicción del modelo. En el caso de p.HER2 parece que la distribución si es normal y que no hay estos valores sobrerepresentados ni extremos.

In [ ]:
#Análisis de las variables discretas
fig, axes = plt.subplots(1,3, figsize=(30,3))
variables=['cell_line', 'time', 'treatment']
for col, ax in enumerate(axes.flatten()):
    sns.countplot(data=cell_phospho, x=variables[col], ax=ax)
    if col==0:
        ax.set_xticklabels(ax.get_xticklabels(), rotation=90, fontsize=8)
    ax.set_title(variables[col])
    ax.set_yticks([])
    ax.set_xlabel(None)
    ax.set_ylabel(None)

Podemos analizar aquí las variables discretas que se utilizarán en el entrenamiento de los datos, podemos observar que hay 44 tipos celulares distintos, que el tiempo transcurre entre 0 y 60 y que hay 6 tipos de tratamiento distintos, vamos a comparar esto con el dataframe de las muestras en las cuales tendremos que predecir los resultados, para hacer frente a cómo vamos a realizar el preprocesamiento de los datos.

In [ ]:
#Análisis de las variables discretas del dataframe que hay que predecir
fig, axes = plt.subplots(1,3, figsize=(30,3))
variables=['cell_line', 'time', 'treatment']
for col, ax in enumerate(axes.flatten()):
    sns.countplot(data=df_challenge1, x=variables[col], ax=ax)
    if col==0:
        ax.set_xticklabels(ax.get_xticklabels(), rotation=90, fontsize=8)
    ax.set_title(variables[col])
    ax.set_yticks([])
    ax.set_xlabel(None)
    ax.set_ylabel(None)

Podemos observar que las lineas celulares son distintas, como sabíamos antes de comenzar el reto, los tratamientos son los mismos y los tiempos concretos la mayoría son los mismos, pero algunos de los presentes en el dataframe de entrenamiento no están presentes en el dataframe que utilizaremos para predecir, esto puede ser un porblema, sobre todo a la hora de que vamos a categorizar la variable tiempo como una variable discreta ordenal, vamos a analizar los datos correspondientes a estos tiempos que solo están presentes en el dataframe cell_phospho.

In [ ]:
strange_times=cell_phospho.query("time == 12.0 or time == 15.0 or time == 16.0 or time == 25.0 or time == 35.0")
print(f'Los datos correspondientes a tiempos extraños son un total de {len(strange_times)}')
print(f'correspondiendo con un {len(strange_times)/len(cell_phospho)*100}% del total')

El análisis de estos valores atípicos muestran que 108712 muestras presentan valores en estos tiempos atípicos, solo correspondiendo con un 0.6% de los datos totales de los que disponemos para entrenamiento. Una opción sería eliminarlos y no utilizarlos en el entrenamiento. Pero la opción que vamos a utilizar es sustituir esos datos por el más cercano, por ejemplo, los 12 se sustituiran por 13, los 15 por 14, los 16 por 17, los 25 por 23 y en el caso de los 35, como ya serán un número muy reducido y están justo entre los tiempos 30 y 40, los eliminaremos para evitar aumentar el ruido.

In [ ]:
import os
# Si no se han procesado el dataframe lo procesa y crea el archivo.
if not os.path.isfile("cell_phospho.csv"):
    cell_phospho=cell_phospho.replace({12.0: 13.0, 15.0: 14.0, 16.0:17.0, 25.0:23.0})
    cell_phospho=cell_phospho.drop(cell_phospho.query("time == 35.0").index)
    cell_phospho.to_csv("cell_phospho.csv")
# Si se habían procesado anteriormente, y el archivo está disponible, lo lee. 
else:
    cell_phospho=pd.read_csv("cell_phospho.csv")

In [ ]:
dis_df_columns = cell_phospho.select_dtypes(exclude=np.number).columns
num_df_columns = cell_phospho.select_dtypes(include=np.number).columns
#Separamos time que lo utilizaremos como una variable discreta ordinal y las usadas en predicción
num_df_columns = num_df_columns.drop(["time", "p.ERK", "p.Akt.Ser473.", "p.S6", "p.HER2", "p.PLCg2"])

In [ ]:
#Analizamos la correlación entre las variables numéricas
cell_phospho[num_df_columns].corr().style.background_gradient()

Podemos ver que entre muchas de las variables existe una correlación bastante buena, superior a 0.5, eso es bueno y también es comprensible, muchas de estas rutas de señalización se encuentran entrelazadas las unas con las otras y cambios en unas afectarán a las demás.

In [ ]:
#Extraemos las columnas de los valores que vamos a intentar predecir con el modelo
p_ERK=cell_phospho.pop("p.ERK")
p_AktSer473=cell_phospho.pop("p.Akt.Ser473.")
p_S6=cell_phospho.pop("p.S6")
p_HER2=cell_phospho.pop("p.HER2")
p_PLCg2=cell_phospho.pop("p.PLCg2")

En este momento podemos comenzar ya con el procesamiento de los datos, para ello vamos a hacer un pipeline de sckitlearn en el que implementemos la transformación de las columnas y la imputación de valores perdidos, antes de nada vamos a analizar qué columnas tienen valores perdidos y cuantos valores perdidos hay en dichas columnas.

In [ ]:
cell_phospho.isna().sum()

In [ ]:
cell_phospho.info()

Parece que no hay 'NaN' en el  dataframe lo cual hace más sencillo el análisis y además vemos que todas las columnas son 'float64' con excepción de treatment y cell_line.

In [ ]:
'''separamos time de las demás variables numéricas ya que la utilizaremos como una variable ordinal, 
el resto utilizaremos OneHotEncoder. Las variables numéricas las procesaremos como StandardScaler'''


from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

#Lista completa de todas las líneas celulares, tanto las que están en entrenamiento como en predicción
cell_lines=[cell_phospho["cell_line"].unique().tolist() + df_challenge1["cell_line"].unique().tolist()]
#Lista de tiempos que se usaran en el ensayo
time_cat = [[0.0, 5.5, 7.0, 9.0, 13.0, 14.0, 17.0, 18.0, 23.0, 30.0, 40.0, 60.0]]
time_transformer = Pipeline([('Ordinal_encoder', OrdinalEncoder(categories=time_cat))])
cell_transformer = Pipeline([('onehot_cell', OneHotEncoder(categories=cell_lines))])
treatment_transformer = Pipeline([('onehot', OneHotEncoder())])
num_transformer = Pipeline([('scaler', StandardScaler())])

phospho_trans = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_df_columns),
        ('cat', cell_transformer, ["cell_line"]),
        ('treat', treatment_transformer, ["treatment"]),
        ('time', time_transformer, ['time'])])

Ahora vamos a proceder a separar los datos de validación y test, lo normal sería aleatorizar las muestras del dataframe para así obtener datos de todas las lineas celulares tanto en el subconjunto de entrenamiento como en el de test, pero, como este modelo está pensado para ser entrenado con los datos de unas lineas celulares y luego utilizarse en el modelado de otras lineas celulares, creo que podría ser una mejor opción, a la hora de no sobreestimar la performance del modelo durante el entrenamiento, separar 7 lineas celulares que se utilizarán para test (aproximadamente el 15% de los datos) y el resto (37 lineas) se utilizarán para entrenamiento.

In [ ]:
test_size=(max(cell_phospho.query("cell_line == 'HCC1187'").index) + 1) #7 primeras lineas celulares para test
phospho_test=cell_phospho.iloc[:test_size].copy()
phospho_train=cell_phospho.iloc[test_size:].copy()

#Lo mismo con las sucesivas variables de interes
p_ERK_test=p_ERK.iloc[:test_size].copy()
p_ERK_train=p_ERK.iloc[test_size:].copy()
p_AktSer473_test=p_AktSer473.iloc[:test_size].copy()
p_AktSer473_train=p_AktSer473.iloc[test_size:].copy()
p_S6_test=p_S6.iloc[:test_size].copy()
p_S6_train=p_S6.iloc[test_size:].copy()
p_HER2_test=p_HER2.iloc[:test_size].copy()
p_HER2_train=p_HER2.iloc[test_size:].copy()
p_PLCg2_test=p_PLCg2.iloc[:test_size].copy()
p_PLCg2_train=p_PLCg2.iloc[test_size:].copy()

Aunque sea más laborioso, se realizarán modelos diferentes para intentar predecir cada una de las variables por separado, así podremos utilizar para cada uno de ellos el acercamiento o el modelo que obtenga mejores resultados.

<a id="section12"></a>
## <font color="#00586D"> 2.3 Modelo predictivo de los niveles de ERK fosforilado. </font>
<br>

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error

# DataFrame con los resultados
results = pd.DataFrame(columns=['Modelo','$R^2$ Entrenamiento','$R^2$ Validación','$R^2$ Test', 
                                         'MAE Entrenamiento', 'MAE Test']).set_index('Modelo')

def show_results(description, model, X_train, y_train, X_test, y_test, is_log=False):
    train_r2 = model.score(X_train,y_train)
    val_r2 = cross_val_score(model,X_train,y_train,cv=5).mean()
    test_r2 = model.score(X_test, y_test)
    # Si se utiliza el logaritmo de la variable objetivo hay que 
    # convertirlo para calcular el error
    f = np.exp if is_log else lambda y: y
    # Calcula el error    
    train_mae = mean_absolute_error(f(y_train), f(model.predict(X_train)))
    test_mae = mean_absolute_error(f(y_test), f(model.predict(X_test)))
    # Muetra los resultados en formato legible
    print('Training \t\txValidation \t\tTest')
    print('-------- \t\t----------- \t\t----')
    print(f"R\u00B2 = {train_r2:.3f}\t\tR\u00B2 = {val_r2:.3f}\t\tR\u00B2 = {test_r2:.3f}")
    print(f"MAE = {train_mae:.2f}\t\t\t\t\tMAE = {test_mae:.2f}")
    results.loc[description]= (train_r2, val_r2, test_r2, train_mae, test_mae)

In [ ]:
#Comenzaremos con un modelo sencillo de regresión lineal de Ridge
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

phospho_pipe_linr = Pipeline([('prep', phospho_trans),
                 ('clas', Ridge())])
parameters = {'clas__alpha':np.logspace(-4, 4, 9, endpoint=True)}

GS = GridSearchCV(phospho_pipe_linr, param_grid=parameters, cv=5)
GS.fit(phospho_train, p_ERK_train)
        
print("Mejor score: ", GS.best_score_)
print("Mejore configuración de parámetros: ", GS.best_params_)

ridge_ERK = GS.best_estimator_

description = 'Ridge regression para ERK'
show_results(description, ridge_ERK, phospho_train, p_ERK_train, phospho_test, p_ERK_test)

In [ ]:
#Guardar el modelo y los resultados
import pickle
filename = 'Models/Ridge_ERK.sav'
pickle.dump(ridge_ERK, open(filename, 'wb'))
results.to_csv("results.csv")
results

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
phospho_pipe_dect = Pipeline([('prep', phospho_trans),
                 ('clas', DecisionTreeRegressor())])
parameters = {'clas__max_depth':np.linspace(1,15,15), 'clas__min_samples_split':(2,4,6,8,10)}
GS = GridSearchCV(phospho_pipe_dect, param_grid=parameters, cv=5)
GS.fit(phospho_train, p_ERK_train)
        
print("Mejor score: ", GS.best_score_)
print("Mejore configuración de parámetros: ", GS.best_params_)

tree_ERK = GS.best_estimator_
description = 'Regression tree for ERK'
show_results(description, tree_ERK, phospho_train, p_ERK_train, phospho_test, p_ERK_test)

In [ ]:
#Guardar el modelo y los resultados
import pickle
filename = 'Models/DecTree_ERK.sav'
pickle.dump(tree_ERK, open(filename, 'wb'))
results.to_csv("results.csv")
results

In [ ]:
from sklearn.ensemble import RandomForestRegressor
phospho_pipe_RF = Pipeline([('prep', phospho_trans),
                 ('clas', RandomForestRegressor())])
parameters = {'clas__max_depth':[1,5,10,15,20]}
GS = GridSearchCV(phospho_pipe_RF, param_grid=parameters, cv=5)
GS.fit(phospho_train, p_ERK_train)
        
print("Mejor score: ", GS.best_score_)
print("Mejore configuración de parámetros: ", GS.best_params_)

RF_ERK = GS.best_estimator_
description = 'Random forest for ERK'
show_results(description, RF_ERK, phospho_train, p_ERK_train, phospho_test, p_ERK_test)

In [ ]:
#Guardar el modelo y los resultados
import pickle
filename = 'Models/RF_ERK.sav'
pickle.dump(RF_ERK, open(filename, 'wb'))
results.to_csv("results.csv")
results

In [ ]:
#función para entrenar y guardar los modelos de Ridge, DecissionTree y RandomForest para cada proteína
def train_models(prot, trains, tests, n):
    #Ridge
    parameters = {'clas__alpha':np.logspace(-4, 4, 5, endpoint=True)}
    GS = GridSearchCV(phospho_pipe_linr, param_grid=parameters, cv=5)
    GS.fit(phospho_train, trains)
    globals()[f"ridge_{prot}"] = GS.best_estimator_
    description = str('Ridge regression para ' + prot)
    show_results(description, globals()[f"ridge_{prot}"], phospho_train, trains, phospho_test, tests)
    #guardar modelo
    filename = str('Models/Ridge_' + prot + '.sav')
    pickle.dump(globals()[f"ridge_{prot}"], open(filename, 'wb'))
    results.to_csv("results.csv")
    #Decision tree
    parameters = {'clas__max_depth':np.linspace(1,15,15), 'clas__min_samples_split':(2,4,6,8,10)}
    GS = GridSearchCV(phospho_pipe_dect, param_grid=parameters, cv=5)
    GS.fit(phospho_train, trains)
    globals()[f"tree_{prot}"] = GS.best_estimator_
    description = str('Decission tree para ' + prot)
    show_results(description, globals()[f"tree_{prot}"], phospho_train, trains, phospho_test, tests)
    #guardar modelo
    filename = str('Models/DecTree_' + prot + '.sav')
    pickle.dump(globals()[f"tree_{prot}"], open(filename, 'wb'))
    results.to_csv("results.csv")
    #Random forest
    parameters = {'clas__max_depth':[1,5,10,15,20]}
    GS = GridSearchCV(phospho_pipe_RF, param_grid=parameters, cv=5)
    GS.fit(phospho_train, trains)
    globals()[f"RF_{prot}"] = GS.best_estimator_
    description = str('Random forest for' + prot)
    show_results(description, globals()[f"RF_{prot}"], phospho_train, trains, phospho_test, tests)
    filename = str('Models/RF_' + prot + '.sav')
    pickle.dump(globals()[f"RF_{prot}"], open(filename, 'wb'))
    results.to_csv("results.csv")

In [6]:
#Entrenar los modelos para todas las proteínas
proteins=["p.Akt.Ser473.", "p.S6", "p.HER2", "p.PLCg2"]
train=["p_AktSer473_train", "p_S6_train", "p_HER2_train", "p_PLCg2_train"]
test=["p_AktSer473_test", "p_S6_test", "p_HER2_test", "p_PLCg2_test"]

for n in range(4):
    train_models(proteins[n], train[n], test[n], n)

p.Akt.Ser473. p_AktSer473_train p_AktSer473_test
p.S6 p_S6_train p_S6_test
p.HER2 p_HER2_train p_HER2_test
p.PLCg2 p_PLCg2_train p_PLCg2_test
